# BASELINE MODEL

Using [this webpage](https://medium.com/@pallawi.ds/ai-starter-train-and-test-your-first-neural-network-classifier-in-keras-from-scratch-b6a5f3b3ebc4) to help me out

Also using [this keras autoencoders guid](https://blog.keras.io/building-autoencoders-in-keras.html)

They are rarely used in practical applications. In 2012 they briefly found an application in greedy layer-wise pretraining for deep convolutional neural networks [1], but this quickly fell out of fashion as we started realizing that better random weight initialization schemes were sufficient for training deep networks from scratch. In 2014, batch normalization [2] started allowing for even deeper networks, and from late 2015 we could train arbitrarily deep networks from scratch using residual learning [3].

# import statements

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
import numpy as np
import random

# import keras
import keras
from keras.models import Sequential
from keras import backend as K
backend_keras = keras.backend.backend()
print("keras is using", backend_keras,"as the backend")
from keras.optimizers import SGD
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D

# imports for autoencoder
from keras.models import Model

Using TensorFlow backend.


keras is using tensorflow as the backend


# Load the data

In [2]:
images = np.load("big_mix_data_6528.npy")
labels_all = np.load("big_mix_labels_6528.npy")
print(images.shape)
print(labels_all.shape)

images = list(images)
labels_all = list(labels_all)

(6528, 200, 200)
(6528, 2)


# Preprocessing

In [3]:
# Shuffle the data
shuffled_images = []
shuffled_labels_all = []
while images:
    i = random.randrange(len(images))
    shuffled_images.append(images[i])
    del images[i]
    shuffled_labels_all.append(labels_all[i])
    del labels_all[i]
    
images = np.array(shuffled_images[:])
labels_all = np.array(shuffled_labels_all)

In [4]:
print(images.shape)
print(labels_all.shape)

(6528, 200, 200)
(6528, 2)


### make the labels so that the model can learn them
Sofar the labels are floats 

### normalise the images
because the values of the pixels in these images are somewhat unconventional I normalize by dividing by 70 instead of 255, this is because I checked and the max value was 66 point something.

In [5]:
# normalize images
images_n = images / 70.0 # n for normalized
print("max : ",np.max(images_n))
print("min : ",np.min(images_n))
images.shape

max :  0.9526767
min :  0.0


(6528, 200, 200)

## Reshape the images for training

In [6]:
im_reshape = np.array(images_n[:])
im_reshape = np.expand_dims(im_reshape, axis=3)

In [7]:
im_reshape = np.array(im_reshape)
print("this is the shape",im_reshape.shape)

this is the shape (6528, 200, 200, 1)


# Train Test Split

In [8]:
# actually i'm only going to do one cut
r1,r2 = 0.75,0.95
cut1 = int(len(im_reshape)*r1)
cut2 = int(len(im_reshape)*r2)
x_train = im_reshape[:cut1]
x_val = im_reshape[cut1:cut2]
x_test = im_reshape[cut2:]


# Define Architecture

In [9]:
input_img = Input(shape=(200, 200, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (25, 25, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = Model(input_img, decoded)

In [45]:
# chose the parameters and compile the model

opt = keras.optimizers.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, amsgrad=True) # adams optimizer
#opt= keras.optimizers.Adagrad(learning_rate=0.01)
#opt = keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
#opt = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)


autoencoder.compile(optimizer=opt, 
                    loss='binary_crossentropy')

print("printing summary of model")
autoencoder.summary()

printing summary of model
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 8)       1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 8)         584       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (Non

# Train the network

In [46]:
from keras.callbacks import TensorBoard
EPOCHS = 5

H = autoencoder.fit(x_train,x_train,
                   epochs=EPOCHS,
                   batch_size=128,
                   shuffle=True,
                   validation_data=(x_val,x_val))


Train on 4896 samples, validate on 1305 samples
Epoch 1/1
  64/4896 [..............................] - ETA: 3:17 - loss: 0.4514

KeyboardInterrupt: 

# OR load the network

In [21]:
#autoencoder = load_model("autoencoder_adams_70.model")

## Evaluate the netork

In [ ]:
print("[INFO] evaluating netowrk...")
decoded_imgs = autoencoder.predict(x_test)

n = 5
plt.figure(figsize=(15,8))
for i in range(n):
    #disp original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(200,200))
    plt.plasma()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstruction
    ax = plt.subplot(2, n, i+n+1)
    plt.imshow(decoded_imgs[i].reshape(200,200))
    plt.plasma()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# plot the training loss and accuracy for each epoch
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
#plt.plot(N, H.history["accuracy"], label="train_acc")
#plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy - 95 to 100")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("training_performance_100.png")
plt.show()

### Save the weights

In [ ]:
# save the model to the disk
print("[INFO] serializing network...")
autoencoder.save("autoencoder_adam_100.model")